##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Red neuronal convolucional (CNN)

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

En este tutorial se muestra el entrenamiento de una sencilla [Red neuronal convolucional](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) para clasificar [imágenes CIFAR](https://www.cs.toronto.edu/~kriz/cifar.html). Dado que este tutorial utiliza la [API secuencial de Keras](https://www.tensorflow.org/guide/keras/overview), crear y entrenar su modelo le llevará tan solo unas pocas líneas de código.


### Importar TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Descargue y prepare el conjunto de datos CIFAR10

El conjunto de datos CIFAR10 contiene 60,000 imágenes en color en 10 clases, con 6,000 imágenes en cada clase. El conjunto de datos se divide en 50,000 imágenes de entrenamiento y 10,000 imágenes de prueba. Las clases son mutuamente excluyentes y no hay superposición entre ellas.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Verifique los datos

Para verificar que el conjunto de datos parece correcto, vamos a representar las 25 primeras imágenes del conjunto de entrenamiento y a mostrar el nombre de la clase debajo de cada imagen:


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Cree la base convolucional

Las 6 líneas de código siguientes definen la base convolucional utilizando un patrón común: una pila de capas [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) y [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D).

Como entrada, una CNN toma tensores de forma (image_height, image_width, color_channels), ignorando el tamaño del lote. Si no conoce estas dimensiones, color_channels se refiere a (R,G,B). En este ejemplo, configurará su CNN para procesar entradas de forma (32, 32, 3), que es el formato de las imágenes CIFAR. Puede hacerlo pasando el argumento `input_shape` a su primera capa.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Visualicemos la arquitectura de su modelo hasta ahora:

In [ ]:
model.summary()

Arriba, puede ver que la salida de cada capa Conv2D y MaxPooling2D es un tensor 3D de forma (altura, ancho, canales). Las dimensiones de ancho y altura tienden a reducirse conforme se profundiza en la red. El número de canales de salida de cada capa Conv2D se controla mediante el primer argumento (por ejemplo, 32 o 64). Normalmente, cuando el ancho y la altura se encogen, puede permitirse (computacionalmente) agregar más canales de salida en cada capa Conv2D.

### Agregue capas densas en la parte superior

Para completar el modelo, introducirá el último tensor de salida de la base convolucional (de forma (4, 4, 64)) en una o varias capas densas para realizar la clasificación. Las capas densas toman vectores como entrada (que son 1D), mientras que la salida actual es un tensor 3D. Primero, aplanará (o desenrollará) la salida 3D a 1D, y después agregará una o más capas Densas en la parte superior. CIFAR tiene 10 clases de salida, por lo que utilizará una capa Densa final con 10 salidas.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

Esta es la arquitectura completa de su modelo:

In [ ]:
model.summary()

El resumen de la red muestra que las salidas (4, 4, 64) se aplanaron en vectores de forma (1024) antes de pasar por dos capas densas.

### Compile y entrene el modelo

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Evaluación del modelo

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

Su CNN simple ha logrado una precisión en las pruebas de más del 70%. ¡No está mal para unas pocas líneas de código! Para otro estilo de CNN, eche un vistazo al ejemplo [TensorFlow 2 quickstart para expertos](https://www.tensorflow.org/tutorials/quickstart/advanced) que utiliza la API de subclases de Keras y `tf.GradientTape`.